In [1]:
# import logfire

# logfire.install_auto_tracing(modules=['cynde'])
# logfire.configure(pydantic_plugin=logfire.PydanticPlugin(record='all'))
import os
import polars as pl
from typing import List, Optional, Tuple, Generator
import time
from cynde.functional.train.types import PredictConfig, BaseClassifierConfig,StratifiedConfig,Feature,FeatureSet,NumericalFeature, CategoricalFeature,EmbeddingFeature, InputConfig, ClassifierConfig, LogisticRegressionConfig, RandomForestClassifierConfig, HistGradientBoostingClassifierConfig, CVConfig, PipelineResults, PipelineInput, PurgedConfig
from cynde.functional.train.preprocess import convert_utf8_to_enum, check_add_cv_index

from sklearn.pipeline import Pipeline
from cynde.functional.train.train_modal import train_nested_cv_distributed
from cynde.functional.train.train_local import train_nested_cv
from cynde.functional.train.preprocess import preprocess_inputs, get_unique_columns


def load_tggate_grouped_data(data_path: str = r"C:\Users\Tommaso\Documents\Dev\Cynde\cache\tgca_nogen_simplified_smiles_malformer_embeddings_ada02_3large_3small_embeddings_grouped.parquet") -> pl.DataFrame:
    return pl.read_parquet(data_path)
def load_moldes_data(data_path:str = r"C:\Users\Tommaso\Documents\Dev\Cynde\cache\MolDes_3D_TG.tsv") -> pl.DataFrame:
    return pl.read_csv(data_path, separator="\t")

def load_and_prepare_data(data_path_tggate: str = r"C:\Users\Tommaso\Documents\Dev\Cynde\cache\tgca_nogen_simplified_smiles_malformer_embeddings_ada02_3large_3small_embeddings_grouped.parquet",
                           data_path_moldes:str = r"C:\Users\Tommaso\Documents\Dev\Cynde\cache\MolDes_3D_TG.tsv") -> pl.DataFrame:
    dft = load_tggate_grouped_data(data_path_tggate)
    dfm = load_moldes_data(data_path_moldes)
    dfm = dfm.with_columns(pl.lit(True).alias("in_moldes"))
    meta_cols = [col for col in dft.columns if "embeddings" not in col]
    text_embedding_cols = [col for col in dft.columns if "embeddings" in col]
    moldes_columns = [col for col in dfm.columns if col not in dft.columns]
    dfj = dft.join(dfm, on="COMPOUND_NAME", how="left")
    dfj = dfj.filter(~pl.col("in_moldes").is_null())
    dfjj = dfj.select(pl.col(meta_cols),pl.col(text_embedding_cols),pl.concat_list(moldes_columns).alias("COMPOUND_NAME_moldes_embeddings"))
    dfjj = (convert_utf8_to_enum(dfjj, threshold=0.2))
    return check_add_cv_index(dfjj,strict=False)


    



In [2]:
df = load_and_prepare_data()
df

cv_index,COMPOUND_NAME,SACRIFICE_PERIOD,DOSE_LEVEL,num_samples,lesion_prob,weight_mean,target,SMILES_CODE_MoLFormer-XL-both-10pct_embeddings,COMPOUND_NAME_text-embedding-ada-002_embeddings,SMILES_CODE_text-embedding-ada-002_embeddings,SACRIFICE_PERIOD_text-embedding-ada-002_embeddings,DOSE_LEVEL_text-embedding-ada-002_embeddings,DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_text-embedding-3-small_embeddings,SMILES_CODE_text-embedding-3-small_embeddings,SACRIFICE_PERIOD_text-embedding-3-small_embeddings,DOSE_LEVEL_text-embedding-3-small_embeddings,DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-small_embeddings,SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_text-embedding-3-large_embeddings,SMILES_CODE_text-embedding-3-large_embeddings,SACRIFICE_PERIOD_text-embedding-3-large_embeddings,DOSE_LEVEL_text-embedding-3-large_embeddings,DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-large_embeddings,SMILES_CODE_DOSE_LEVEL_text-embedding-3-large_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_LEVEL_text-embedding-3-large_embeddings,COMPOUND_NAME_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_moldes_embeddings
u32,enum,enum,enum,u32,f64,f64,i32,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64]
0,"""cisplatin""","""29 day""","""High""",1,1.0,172.0,1,"[1.173913, 0.555346, … -0.425011]","[-0.024641, -0.00057, … -0.000275]","[0.003289, -0.012578, … -0.033704]","[-0.00606, -0.006973, … -0.04334]","[0.000387, -0.004637, … -0.018972]","[0.014158, -0.003536, … -0.013693]","[-0.014984, -0.003164, … -0.014547]","[0.007863, -0.007235, … -0.024785]","[-0.01225, -0.011

In [3]:


cols = ['COMPOUND_NAME',
        'SACRIFICE_PERIOD',
        'DOSE_LEVEL',
        'num_samples',
        'lesion_prob',
        'weight_mean',
        'SMILES_CODE_MoLFormer-XL-both-10pct_embeddings',
        'COMPOUND_NAME_text-embedding-ada-002_embeddings',
        'SMILES_CODE_text-embedding-ada-002_embeddings',
        'SACRIFICE_PERIOD_text-embedding-ada-002_embeddings',
        'DOSE_LEVEL_text-embedding-ada-002_embeddings',
        'DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings',
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings',
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings',
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings',
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings',
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings',
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings',
        'COMPOUND_NAME_text-embedding-3-small_embeddings', 
        'SMILES_CODE_text-embedding-3-small_embeddings',
        'SACRIFICE_PERIOD_text-embedding-3-small_embeddings', 
        'DOSE_LEVEL_text-embedding-3-small_embeddings',
        'DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-small_embeddings',
        'SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings',
        'COMPOUND_NAME_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings',
        'COMPOUND_NAME_SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_text-embedding-3-large_embeddings',
        'SMILES_CODE_text-embedding-3-large_embeddings', 
        'SACRIFICE_PERIOD_text-embedding-3-large_embeddings', 
        'DOSE_LEVEL_text-embedding-3-large_embeddings',
        'DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-large_embeddings',
        'SMILES_CODE_DOSE_LEVEL_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_DOSE_LEVEL_text-embedding-3-large_embeddings',
        'COMPOUND_NAME_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings', 
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-large_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings', 
        'SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings', 
        'COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-3-small_embeddings', 
        'target']


feature_set_molformer={"embeddings":[{"column_name":"SMILES_CODE_MoLFormer-XL-both-10pct_embeddings",
                                        "name":"embeddings of the compound names",
                                        "embedder": "molformer",
                                        "embedding_size":768},
                                         {"column_name":"DOSE_LEVEL_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the dose levels",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072},
                                            {"column_name":"SACRIFICE_PERIOD_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the sacrifice periods",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072}]}

feature_set_moldes={"embeddings":[{"column_name":"COMPOUND_NAME_moldes_embeddings",
                                        "name":"embeddings of the compound names",
                                        "embedder": "molder",
                                        "embedding_size":1826},
                                         {"column_name":"DOSE_LEVEL_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the dose levels",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072},
                                            {"column_name":"SACRIFICE_PERIOD_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the sacrifice periods",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072}]}

#the best model combines compound_name, dose_level and sacrifice period embeddings each indipendently using hte large embeddings
feature_set_best_model = {"embeddings":[{"column_name":"COMPOUND_NAME_text-embedding-3-large_embeddings",
                                         "name":"embeddings of the compounds names",
                                         "embedder": "text-embedding-3-large_embeddings",
                                         "embedding_size":3072},
                                         {"column_name":"DOSE_LEVEL_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the dose levels",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072},
                                            {"column_name":"SACRIFICE_PERIOD_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the sacrifice periods",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072}]}

feature_set_only_compound_name = {"embeddings":[{"column_name":"COMPOUND_NAME_text-embedding-3-large_embeddings",
                                            "name":"embeddings of the compounds names",
                                            "embedder": "text-embedding-3-large_embeddings",
                                            "embedding_size":3072}]}

input_config_data = {"feature_sets":[feature_set_molformer,feature_set_moldes,feature_set_best_model],
                        "target_column":"target",
                        "save_folder":"C:/Users/Tommaso/Documents/Dev/Cynde/cynde_mount/",
                        "remote_folder":"/cynde_mount"}

classifiers_config = ClassifierConfig(classifiers=[RandomForestClassifierConfig(n_estimators=10, max_depth=5)])

input_config = InputConfig.model_validate(input_config_data,context={"df":df})

inner_groups = ["DOSE_LEVEL","SACRIFICE_PERIOD","target"]
outer_groups = ["COMPOUND_NAME"]

cv_config = CVConfig(inner= StratifiedConfig(groups=inner_groups,k=2),
                     inner_replicas=1,
                     outer = PurgedConfig(groups=outer_groups,k=2),
                        outer_replicas=1)

task = PredictConfig(input_config=input_config, cv_config=cv_config, classifiers_config=classifiers_config)






df_filtered = df.select(get_unique_columns(task))

preprocess_inputs(df_filtered,task.input_config)
results = train_nested_cv(df,task)

# results = train_nested_cv_distributed(df_filtered,task)

#todo
# 1) fix the cv objects for purged (add a test) to the pydantic object I guess
# 3) results saving and aggregation

selected columns: ['cv_index', 'target', 'SMILES_CODE_MoLFormer-XL-both-10pct_embeddings', 'DOSE_LEVEL_text-embedding-3-large_embeddings', 'SACRIFICE_PERIOD_text-embedding-3-large_embeddings']
selected columns: ['cv_index', 'target', 'COMPOUND_NAME_moldes_embeddings', 'DOSE_LEVEL_text-embedding-3-large_embeddings', 'SACRIFICE_PERIOD_text-embedding-3-large_embeddings']
selected columns: ['cv_index', 'target', 'COMPOUND_NAME_text-embedding-3-large_embeddings', 'DOSE_LEVEL_text-embedding-3-large_embeddings', 'SACRIFICE_PERIOD_text-embedding-3-large_embeddings']
using groups ['COMPOUND_NAME', 'DOSE_LEVEL', 'SACRIFICE_PERIOD', 'target'] to generate the cv folds
For outer replica 0, outer fold 0, inner replica 0, inner fold 0: train 415, val 404, test 804 samples.
Training pipeline with classifier RandomForestClassifier on feature set numerical=[] embeddings=[EmbeddingFeature(column_name='SMILES_CODE_MoLFormer-XL-both-10pct_embeddings', name='embeddings of the compound names', description=No

C:\Users\Tommaso\Documents\Dev\Cynde\cynde\functional\train\cv.py:138: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  sliced = sdf.select(pl.col("cv_index").map_elements(lambda s: hacky_list_relative_slice(s,k)).alias("hacky_cv_index")).unnest("hacky_cv_index")


Converting SACRIFICE_PERIOD_text-embedding-3-large_embeddings of type List(Float64) to a list of columns.
shape: (1_623, 6_914)
┌──────────┬────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ cv_index ┆ target ┆ SMILES_COD ┆ SMILES_COD ┆ … ┆ SACRIFICE_ ┆ SACRIFICE ┆ SACRIFICE ┆ SACRIFICE │
│ ---      ┆ ---    ┆ E_MoLForme ┆ E_MoLForme ┆   ┆ PERIOD_tex ┆ _PERIOD_t ┆ _PERIOD_t ┆ _PERIOD_t │
│ u32      ┆ i32    ┆ r-XL-both- ┆ r-XL-both- ┆   ┆ t-embeddin ┆ ext-embed ┆ ext-embed ┆ ext-embed │
│          ┆        ┆ …          ┆ …          ┆   ┆ …          ┆ din…      ┆ din…      ┆ din…      │
│          ┆        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│          ┆        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 0        ┆ 1      ┆ 1.173913   ┆ 0.555346   ┆ … ┆ 0.00799    ┆

C:\Users\Tommaso\Documents\Dev\Cynde\cynde\functional\train\cv.py:138: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  sliced = sdf.select(pl.col("cv_index").map_elements(lambda s: hacky_list_relative_slice(s,k)).alias("hacky_cv_index")).unnest("hacky_cv_index")


Converting SACRIFICE_PERIOD_text-embedding-3-large_embeddings of type List(Float64) to a list of columns.
shape: (1_623, 6_914)
┌──────────┬────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ cv_index ┆ target ┆ SMILES_COD ┆ SMILES_COD ┆ … ┆ SACRIFICE_ ┆ SACRIFICE ┆ SACRIFICE ┆ SACRIFICE │
│ ---      ┆ ---    ┆ E_MoLForme ┆ E_MoLForme ┆   ┆ PERIOD_tex ┆ _PERIOD_t ┆ _PERIOD_t ┆ _PERIOD_t │
│ u32      ┆ i32    ┆ r-XL-both- ┆ r-XL-both- ┆   ┆ t-embeddin ┆ ext-embed ┆ ext-embed ┆ ext-embed │
│          ┆        ┆ …          ┆ …          ┆   ┆ …          ┆ din…      ┆ din…      ┆ din…      │
│          ┆        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│          ┆        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞══════════╪════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 0        ┆ 1      ┆ 1.173913   ┆ 0.555346   ┆ … ┆ 0.00799    ┆

In [5]:
from cynde.functional.train.results import merge_results, merge_metrics,aggr_metrics_by_inputs, save_results
save_results(results,"gang_")


Results saved at C:/Users/Tommaso/Documents/Dev/Cynde/cynde_mount/results_20240517-133316.parquet
Metrics saved at C:/Users/Tommaso/Documents/Dev/Cynde/cynde_mount/metrics_20240517-133316.parquet


In [ ]:
merge_results(results)

In [ ]:
out = merge_metrics(results)

In [ ]:
aggr_metrics_by_inputs(out)

feature_set_name,classifier_id,train_accuracy,train_mcc,val_accuracy,val_mcc,test_accuracy,test_mcc,len
str,str,f64,f64,f64,f64,f64,f64,u32
"""COMPOUND_NAME_text-embedding-3…","""bootstrap_True_ccp_alpha_0.0_c…",0.910045,0.820576,0.798778,0.597823,0.621905,0.245614,4
"""COMPOUND_NAME_moldes_embedding…","""bootstrap_True_ccp_alpha_0.0_c…",0.906984,0.815096,0.785725,0.571586,0.61962,0.243336,4
"""DOSE_LEVEL_text-embedding-3-la…","""bootstrap_True_ccp_alpha_0.0_c…",0.909802,0.821435,0.775696,0.552485,0.633221,0.267879,4


In [ ]:
metrics_columns = ["train_accuracy","train_mcc","val_accuracy","val_mcc","test_accuracy","test_mcc"]

In [ ]:
out.group_by("feature_set_name","classifier_id").agg(pl.mean(metrics_columns),pl.len())

feature_set_name,classifier_id,train_accuracy,train_mcc,val_accuracy,val_mcc,test_accuracy,test_mcc,len
str,str,f64,f64,f64,f64,f64,f64,u32
"""COMPOUND_NAME_moldes_embedding…","""bootstrap_True_ccp_alpha_0.0_c…",0.906984,0.815096,0.785725,0.571586,0.61962,0.243336,4
"""DOSE_LEVEL_text-embedding-3-la…","""bootstrap_True_ccp_alpha_0.0_c…",0.909802,0.821435,0.775696,0.552485,0.633221,0.267879,4
"""COMPOUND_NAME_text-embedding-3…","""bootstrap_True_ccp_alpha_0.0_c…",0.910045,0.820576,0.798778,0.597823,0.621905,0.245614,4
